<a href="https://colab.research.google.com/github/Yashrjadhav/2D23D/blob/main/TripoNEW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/VAST-AI-Research/TripoSR.git
%cd TripoSR

Cloning into 'TripoSR'...
remote: Enumerating objects: 161, done.
remote: Counting objects: 100% (79/79), done.
remote: Compressing objects: 100% (48/48), done.
remote: Total 161 (delta 50), reused 31 (delta 31), pack-reused 82 (from 1)
Receiving objects: 100% (161/161), 36.71 MiB | 21.08 MiB/s, done.
Resolving deltas: 100% (63/63), done.
/content/TripoSR


In [ ]:
!pip install -r requirements.txt

  Cloning https://github.com/tatsy/torchmcubes.git to /tmp/pip-req-build-kdmfu1un
  Running command git clone --filter=blob:none --quiet https://github.com/tatsy/torchmcubes.git /tmp/pip-req-build-kdmfu1un
  Resolved https://github.com/tatsy/torchmcubes.git to commit 3381600ddc3d2e4d74222f8495866be5fafbace4
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.1/123.1 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 11.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of tokenizers to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of tokenizers to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might nee

In [ ]:
%cd /content/TripoSR


/content/TripoSR


In [ ]:
!pip install onnxruntime
!pip install rembg torch xatlas pillow numpy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 100.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.0 MB/s eta 0:00:00


In [ ]:
!pip install --upgrade huggingface_hub transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.4/481.4 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 85.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 71.4 MB/s eta 0:00:00
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.17.3
    Uninstalling huggingface-hub-0.17.3:
      Successfully uninstalled huggingface-hub-0.17.3
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.14.1
    Uninstalling tokenizers-0.14.1:
      Successfully uninstalled tokenizers-0.14.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.0
    Uninstalling transformers-4.35.0:
      Successfully uninstalled transformers-4.35.0


In [ ]:
# STEP 1: Install required libraries
!pip install pyngrok flask-ngrok

# STEP 2: Set up ngrok auth token
from pyngrok import ngrok
ngrok.set_auth_token("2sikRPWvncEuMMu4MTgloVwsCVx_4keXjxWApcS3JXmjcf2ct")  # Replace with your token

# STEP 3: Imports and App Setup
from flask import Flask, request, jsonify, send_from_directory
import os
import subprocess
import threading
import uuid
import time

app = Flask(__name__)
UPLOAD_FOLDER = '/content/uploads'
OUTPUT_FOLDER = '/content/output'
os.makedirs(UPLOAD_FOLDER, exist_ok=True)
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# STEP 4: Ngrok Public URL (extract the public URL string)
tunnel = ngrok.connect(5000)
public_url = tunnel.public_url
print(f"\n🚀 Public API endpoint: {public_url}/convert")

# STEP 5: Dummy root endpoint
@app.route('/')
def root():
    return jsonify({"message": "Server is running. Use /convert to post your image."})

# STEP 6: Conversion Logic
def convert_image_to_3d_glb(input_image_path, job_id):
    output_dir = f"{OUTPUT_FOLDER}/{job_id}"
    os.makedirs(output_dir, exist_ok=True)
    command = [
        'python', '/content/TripoSR/run.py',  # Make sure this file exists and works
        input_image_path,
        '--device', 'cuda:0',
        '--output-dir', output_dir,
        '--model-save-format', 'glb'
    ]
    subprocess.run(command, check=True)
    return f"{output_dir}/0/mesh.glb"  # Assuming this structure is correct

# STEP 7: Upload and Convert Endpoint
@app.route('/convert', methods=['POST'])
def convert():
    if 'image' not in request.files:
        return jsonify({'error': 'No image file found'}), 400

    image = request.files['image']
    job_id = str(uuid.uuid4())[:8]
    input_path = os.path.join(UPLOAD_FOLDER, f"{job_id}.jpg")
    image.save(input_path)
    print(f"📸 Image saved: {input_path}, Job ID: {job_id}")

    try:
        glb_path = convert_image_to_3d_glb(input_path, job_id)
    except subprocess.CalledProcessError as e:
        print(f"❌ Conversion failed: {e}")
        return jsonify({'error': '3D conversion failed', 'details': str(e)}), 500

    glb_filename = os.path.basename(glb_path)
    glb_url = f"{public_url}/download/{job_id}/0/{glb_filename}"
    print(f"✅ Conversion done! File URL: {glb_url}")
    return jsonify({'glb_url': glb_url})

# STEP 8: File Download Endpoint
@app.route('/download/<job_id>/<subfolder>/<filename>')
def download_glb(job_id, subfolder, filename):
    directory = f"{OUTPUT_FOLDER}/{job_id}/{subfolder}"
    print(f"📤 Sending: {directory}/{filename}")
    return send_from_directory(directory, filename)

# STEP 9: Run Flask in background thread
def run_app():
    app.run(host="0.0.0.0", port=5000, use_reloader=False)

# STEP 10: Start server and keep cell alive
flask_thread = threading.Thread(target=run_app)
flask_thread.start()

# STEP 11: Keep the cell alive
try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    print("🛑 Server stopped.")



🚀 Public API endpoint: https://95ab-104-196-139-240.ngrok-free.app/convert
 * Serving Flask app '__main__'
 * Debug mode: off


Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.
INFO:werkzeug:127.0.0.1 - - [12/Apr/2025 07:31:39] "POST /convert HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [12/Apr/2025 08:03:12] "POST /convert HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [12/Apr/2025 08:10:50] "POST /convert HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [12/Apr/2025 08:13:39] "GET /download/4154bbd0/0/mesh.glb HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [12/Apr/2025 08:15:47] "GET /download/4154bbd0/0/mesh.glb HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [12/Apr/2025 08:16:24] "GET /download/4154bbd0/0/mesh.glb HTTP/1.1" 304 -


In [ ]:
!python /content/TripoSR/run.py /content/TripoSR/examples/hamburger.png --output-dir output/ --device cuda:0 --model-save-format glb

2025-04-06 09:45:18,424 - INFO - Initializing model ...
config.yaml: 100% 987/987 [00:00<00:00, 6.90MB/s]
model.ckpt: 100% 1.68G/1.68G [00:07<00:00, 230MB/s]
2025-04-06 09:45:30,252 - INFO - NumExpr defaulting to 2 threads.
2025-04-06 09:45:35.597218: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743932735.848610    8321 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743932735.918123    8321 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-06 09:45:36.462573: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following in

In [ ]:
!python /content/TripoSR/run.py /content/TripoSR/examples/shoe3.png --device cuda:0 --output-dir /content/output/

Traceback (most recent call last):
  File "/content/TripoSR/run.py", line 8, in <module>
    import torch
  File "/usr/local/lib/python3.11/dist-packages/torch/__init__.py", line 404, in <module>
    _load_global_deps()
  File "/usr/local/lib/python3.11/dist-packages/torch/__init__.py", line 334, in _load_global_deps
    _preload_cuda_deps("nvjitlink", "libnvJitLink.so.*[0-9]")
  File "/usr/local/lib/python3.11/dist-packages/torch/__init__.py", line 303, in _preload_cuda_deps
    ctypes.CDLL(lib_path)
  File "/usr/lib/python3.11/ctypes/__init__.py", line 376, in __init__
    self._handle = _dlopen(self._name, mode)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt
^C


In [ ]:
#signup to ngrok.com and copy the authtoken generated/present there and paste here
!pip install pyngrok flask-ngrok
!ngrok config add-authtoken 2sikRPWvncEuMMu4MTgloVwsCVx_4keXjxWApcS3JXmjcf2ct
from pyngrok import ngrok
ngrok.set_auth_token("2sikRPWvncEuMMu4MTgloVwsCVx_4keXjxWApcS3JXmjcf2ct")



Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from flask import Flask, request, send_file, jsonify
import os
import subprocess
import threading
from pyngrok import ngrok

app = Flask(__name__)

# Set your ngrok auth token (if not already done in a previous cell)
ngrok.set_auth_token("2sikRPWvncEuMMu4MTgloVwsCVx_4keXjxWApcS3JXmjcf2ct")

# Manually open a tunnel on port 5000
public_url = ngrok.connect(5000)
print("Public URL:", public_url)

def convert_image_to_3d_model(input_image_path, output_model_path):
    output_dir = '/content/output/'
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    # Build the command to run your conversion pipeline (run.py)
    command = [
        'python', '/content/TripoSR/run.py',
        input_image_path,
        '--device', 'cuda:0',
        '--output-dir', output_dir
    ]
    subprocess.run(command, check=True)

@app.route('/convert', methods=['POST'])
def convert():
    if 'image' not in request.files:
        return jsonify({'error': 'No image provided'}), 400

    image_file = request.files['image']
    input_path = 'temp_input.jpg'
    image_file.save(input_path)
    # According to your run.py, for the first image the mesh is saved in /content/output/0/mesh.obj
    output_model = '/content/output/0/mesh.obj'

    try:
        convert_image_to_3d_model(input_path, output_model)
    except subprocess.CalledProcessError as e:
        return jsonify({'error': 'Conversion pipeline failed', 'details': str(e)}), 500

    os.remove(input_path)
    return send_file(output_model, as_attachment=True)

# Run the Flask server in a background thread so the cell remains live.
threading.Thread(target=app.run, kwargs={'use_reloader': False}).start()


Public URL: NgrokTunnel: "https://9441-34-16-190-172.ngrok-free.app" -> "http://localhost:5000"


In [ ]:
#replca the default api url with the generate public api url from above generated cell
!curl -F "image=@/content/TripoSR/examples/teapot.png" https://9441-34-16-190-172.ngrok-free.app/convert --output output_model.obj


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  269k    0     0  100  269k      0  10871  0:00:25  0:00:25 --:--:--     0

INFO:werkzeug:127.0.0.1 - - [07/Feb/2025 17:09:53] "POST /convert HTTP/1.1" 200 -


100 4646k  100 4376k  100  269k   159k  10065  0:00:27  0:00:27 --:--:-- 1084k
